In [1]:
import pandas as pd
from plotly.offline import iplot
import plotly.graph_objs as go
import numpy as np
from sklearn import metrics
import math
import matplotlib.pyplot as plt
from collections import defaultdict
from functools import partial
from itertools import repeat
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.metrics import roc_curve, auc
from keras.utils import to_categorical
from keras import models
from keras import layers
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.layers import Dense, Dropout, Activation
import keras.backend as K
import os
from keras import metrics

Using TensorFlow backend.


In [3]:
# returns dataframe
# params: full -> True = full dataset, False = first file

def grab_data(full, path):
    if full:
        return pd.concat([pd.read_csv(path+file, index_col=False) for file in os.listdir(path)])
    else:
        for file in os.listdir(path):
            if "10_1_" not in file: continue
            return pd.read_csv(path+file, index_col=False)

In [4]:
# import data
nBodies = 3
workDir = "/Users/brandonmanley/Documents/nBody/data"
dataPath = "/Users/brandonmanley/Documents/nBody/data/brutusSim/"+str(nBodies)+"body/"
        
df = grab_data(False, dataPath)
df.head()

,finalFile,eventID,m1,m2,m3,x1,x2,x3,y1,y2,...,xf3,yf1,yf2,yf3,dxf1,dxf2,dxf3,dyf1,dyf2,dyf3
0,1,10001,84.270385,56.314115,52.279696,-2.226488,-3.406637,9.440128,-3.887835,-4.902674,...,9.442821,-3.888589,-4.899092,-2.849904,0.457428,-0.525155,0.687588,-0.222422,0.961199,-0.100621
1,1,10002,84.270385,56.314115,52.279696,-2.226488,-3.406637,9.440128,-3.887835,-4.902674,...,9.445500,-3.889573,-4.895165,-2.850298,0.389737,-0.420452,0.683916,-0.281080,1.049364,-0.101038
2,1,10003,84.270385,56.314115,52.279696,-2.226488,-3.406637,9.440128,-3.887835,-4.902674,...,9.448164,-3.890785,-4.890894,-2.850693,0.321736,-0.315285,0.680246,-0.339542,1.137235,-0.101454
3,1,10004,84.270385,56.314115,52.279696,-2.226488,-3.406637,9.440128,-3.887835,-4.902674,...,9.450814,-3.892225,-4.886281,-2.851090,0.253345,-0.209536,0.676576,-0.397869,1.224904,-0.101870
4,1,10005,84.270385,56.314115,52.279696,-2.226488,-3.406637,9.440128,-3.887835,-4.902674,...,9.453450,-3.893893,-4.881325,-2.851489,0.184487,-0.103088,0.672908,-0.456125,1.312466,-0.102286


In [5]:
# INPUT: want mass, x/y pos, dx/dy for n bodies -> total input is n*5 + 1 (time)
# OUTPUT: want x/y, dx/dy -> total input is n*4


i_col, o_col = [], []
colNames = ["m", "x", "y", "dx", "dy"]

for col in colNames:
    for n in range(1, nBodies+1):
        i_col.append(col+str(n))
        if col != "m":
            o_col.append(col+"f"+str(n))
i_col.append("t")
    
X1 = df.as_matrix(columns=i_col)
y1 = df.as_matrix(columns=o_col)

print(X1.shape, y1.shape)

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:15: FutureWarning:

Method .as_matrix will be removed in a future version. Use .values instead.

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:16: FutureWarning:

Method .as_matrix will be removed in a future version. Use .values instead.



(1434379, 16) (1434379, 12)


In [7]:
# make predictions
loss_function = 'mse'
my_metrics = ['mae']

network = models.Sequential()
network.add(layers.Dense(128, activation='relu', input_dim=X1.shape[1]))
for i in range(9):
    network.add(layers.Dense(128, activation='relu'))
network.add(layers.Dense(y1.shape[1],activation='linear'))
network.compile(optimizer='adam', loss=loss_function, metrics=my_metrics)
network.load_weights(workDir + '/weights/final_{0}body.h5'.format(nBodies))
yPred = network.predict(X1)

In [8]:
# save predictions
dfPred = pd.DataFrame(yPred, index=None, columns=[o+"p" for o in o_col])
for col in dfPred.columns:
    df[col] = dfPred[col]
df.to_csv(workDir+"/pred/comb10_1_{0}.csv".format(nBodies), index=False)

In [9]:
df

,finalFile,eventID,m1,m2,m3,x1,x2,x3,y1,y2,...,xf3p,yf1p,yf2p,yf3p,dxf1p,dxf2p,dxf3p,dyf1p,dyf2p,dyf3p
0,1,10001,84.270385,56.314115,52.279696,-2.226488,-3.406637,9.440128,-3.887835,-4.902674,...,9.923834,-4.641181,-5.110014,-3.325714,0.247840,0.454010,0.021056,-0.149905,0.830006,-0.608818
1,1,10002,84.270385,56.314115,52.279696,-2.226488,-3.406637,9.440128,-3.887835,-4.902674,...,9.922998,-4.639993,-5.106413,-3.325877,0.247676,0.456748,0.018070,-0.149637,0.832570,-0.611098
2,1,10003,84.270385,56.314115,52.279696,-2.226488,-3.406637,9.440128,-3.887835,-4.902674,...,9.922163,-4.638801,-5.102811,-3.326045,0.247503,0.459490,0.015086,-0.149364,0.835131,-0.613381
3,1,10004,84.270385,56.314115,52.279696,-2.226488,-3.406637,9.440128,-3.887835,-4.902674,...,9.921320,-4.637608,-5.099202,-3.326202,0.247338,0.462221,0.012100,-0.149093,0.837693,-0.615664
4,1,10005,84.270385,56.314115,52.279696,-2.226488,-3.406637,9.440128,-3.887835,-4.902674,...,9.920257,-4.636385,-5.095592,-3.326385,0.247127,0.465205,0.008975,-0.148602,0.840033,-0.617980
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1434374,1,6122556,84.213464,69.451447,7.514575,-2.270110,-3.588650,-0.570330,3.553075,8.649397,...,-10.624702,2.086395,6.294528,10.038875,-2.451386,0.685104,0.920845,-0.166407,0.014975,0.667711
1434375,1,6122557,84.213464,69.451447,7.514575,-2.270110,-3.588650,-0.570330,3.553075,8.649397,...,-10.625870,2.085242,6.296760,10.034710,-2.455737,0.693192,0.918486,-0.169102,0.016365,0.670553
1434376,1,6122558,84.213464,69.451447,7.514575,-2.270110,-3.588650,-0.570330,3.553075,8.649397,...,-10.627009,2.084097,6.298985,10.030555,-2.460074,0.701249,0.916136,-0.171792,0.017759,0.673379
1434377,1,6122559,84.213464,69.451447,7.514575,-2.270110,-3.588650,-0.570330,3.553075,8.649397,...,-10.628168,2.082937,6.301221,10.026399,-2.464414,0.709323,0.913782,-0.174490,0.019149,0.676212
